In [5]:
import re
import pandas as pd
from typing import  Literal,List,Any
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langgraph.types import Command
from langgraph.graph.message import add_messages
from typing_extensions import TypedDict, Annotated
from langchain_core.prompts.chat import ChatPromptTemplate
from langgraph.graph import START, StateGraph,END
from langgraph.prebuilt import create_react_agent
from pydantic import BaseModel, Field, field_validator
from langchain_core.messages import HumanMessage,AIMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [6]:
load_dotenv()

True

In [8]:
import os
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"]=OPENAI_API_KEY

In [9]:
openai_model=ChatOpenAI(model="gpt-4o")

In [11]:
openai_model.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_f33640a400', 'id': 'chatcmpl-CFZw6XNqhussF7xHw8eR6cjh9pZCW', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac24d713-d2b8-469e-8f8a-5d244aed8d03-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [13]:
from data_models.models import DateTimeModel, DateModel, IdentificationNumberModel

## Tools

### Check Availability by Doctor

In [17]:
df = pd.read_csv("..\\data\\doctor_availability.csv")
df.head()

,date_slot,specialization,doctor_name,is_available,patient_to_attend
0,05-08-2024 08:00,general_dentist,john doe,True,NaN
1,05-08-2024 08:30,general_dentist,john doe,False,1000082.0
2,05-08-2024 09:00,general_dentist,john doe,False,1000048.0
3,05-08-2024 09:30,general_dentist,john doe,False,1000036.0
4,05-08-2024 10:00,general_dentist,john doe,False,1000024.0


In [19]:
df['date_slot_time'] = df['date_slot'].apply(lambda input: input.split(" ")[-1])
# df['date_slot'].apply(lambda input: input.split(" ")[-1])
df.head()

,date_slot,specialization,doctor_name,is_available,patient_to_attend,date_slot_time
0,05-08-2024 08:00,general_dentist,john doe,True,NaN,08:00
1,05-08-2024 08:30,general_dentist,john doe,False,1000082.0,08:30
2,05-08-2024 09:00,general_dentist,john doe,False,1000048.0,09:00
3,05-08-2024 09:30,general_dentist,john doe,False,1000036.0,09:30
4,05-08-2024 10:00,general_dentist,john doe,False,1000024.0,10:00


In [ ]:
desired_date = "07-08-2024"
doctor_name = "john doe"

rows = list(df[(df['date_slot'].apply(lambda input: input.split(' ')[0]) == desired_date)&(df['doctor_name'] == doctor_name)&(df['is_available'] == True)]['date_slot_time'])
rows

['08:00',
 '08:30',
 '09:30',
 '10:30',
 '11:00',
 '11:30',
 '13:00',
 '14:00',
 '14:30',
 '15:00',
 '16:00',
 '16:30']

In [28]:
if len(rows) == 0:
    output = "No availability in the entire day"
else:
    output = f'This availability for {desired_date}\n'
    output += "Available slots: " + ', '.join(rows)

output

'This availability for 07-08-2024\nAvailable slots: 08:00, 08:30, 09:30, 10:30, 11:00, 11:30, 13:00, 14:00, 14:30, 15:00, 16:00, 16:30'

In [29]:
@tool
def check_availability_by_doctor(desired_date:DateModel, doctor_name:Literal['kevin anderson','robert martinez','susan davis','daniel miller','sarah wilson','michael green','lisa brown','jane smith','emily johnson','john doe']):
    """
    Checking the database if we have availability for the specific doctor.
    The parameters should be mentioned by the user in the query
    """
    df = pd.read_csv("..\\data\\doctor_availability.csv")
    #print(df)
    
    df['date_slot_time'] = df['date_slot'].apply(lambda input: input.split(' ')[-1])
    
    rows = list(df[(df['date_slot'].apply(lambda input: input.split(' ')[0]) == desired_date.date)&(df['doctor_name'] == doctor_name)&(df['is_available'] == True)]['date_slot_time'])

    if len(rows) == 0:
        output = "No availability in the entire day"
    else:
        output = f'This availability for {desired_date.date}\n'
        output += "Available slots: " + ', '.join(rows)

    return output

In [31]:
# Example usage:
date_instance = DateModel(date="10-08-2024")
print(date_instance)

date='10-08-2024'


In [32]:
print(check_availability_by_doctor.invoke({"desired_date": date_instance, "doctor_name": "emily johnson"}))

This availability for 10-08-2024
Available slots: 09:30, 10:00, 10:30, 11:00, 11:30, 12:00
